### importing packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

sys.path.append(os.path.abspath(os.path.join("../script")))
from utils import percent_missing,format_float, find_agg, missing_values_table,convert_bytes_to_megabytes,fix_missing_ffill,fix_missing_bfill

In [2]:
from sqlalchemy import create_engine

database_name = 'week1'
table_name= 'xdr_data'

connection_params = { "host": "localhost", "user": "postgres", "password": "pgadmin",
                    "port": "5432", "database": database_name}

engine = create_engine(f"postgresql+psycopg2://{connection_params['user']}:{connection_params['password']}@{connection_params['host']}:{connection_params['port']}/{connection_params['database']}")

# str or SQLAlchemy Selectable (select or text object)
sql_query = 'SELECT * FROM xdr_data'

df = pd.read_sql(sql_query, con= engine)

In [3]:
df.columns

Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',


In [4]:
selected_columns = [
    'MSISDN/Number',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
    'Avg Bearer TP UL (kbps)', 'Avg Bearer TP DL (kbps)',
    'Handset Type']

In [5]:
selected_df = df[selected_columns]
selected_df

,MSISDN/Number,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP UL (kbps),Avg Bearer TP DL (kbps),Handset Type
0,3.366496e+10,NaN,NaN,42.0,5.0,44.0,23.0,Samsung Galaxy A5 Sm-A520F
1,3.368185e+10,NaN,NaN,65.0,5.0,26.0,16.0,Samsung Galaxy J5 (Sm-J530)
2,3.376063e+10,NaN,NaN,NaN,NaN,9.0,6.0,Samsung Galaxy A8 (2018)
3,3.375034e+10,NaN,NaN,NaN,NaN,44.0,44.0,undefined
4,3.369980e+10,NaN,NaN,NaN,NaN,9.0,6.0,Samsung Sm-G390F
...,...,...,...,...,...,...,...,...
149996,3.365069e+10,NaN,NaN,32.0,0.0,65.0,52.0,Apple iPhone 8 Plus (A1897)
149997,3.366345e+10,NaN,NaN,27.0,2.0,54.0,23.0,Apple iPhone Se (A1723)
149998,3.362189e+10,NaN,NaN,43.0,6.0,47.0,43.0,Apple iPhone Xs (A2097)
149999,3.361962e+10,NaN,NaN,37.0,5.0,37.0,34.0,Huawei Fig-Lx1


In [7]:
# Assuming 'data' is your DataFrame containing the dataset with the specified columns

# Replace missing values with the mean for numerical columns (TCP retransmission, RTT, throughput)
numeric_columns = ['TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
                   'Avg RTT DL (ms)', 'Avg RTT UL (ms)',
                   'Avg Bearer TP UL (kbps)', 'Avg Bearer TP DL (kbps)']

for col in numeric_columns:
    df[col].fillna(df[col].mean(), inplace=True)

# Replace missing handset type values with the mode (most frequent)
df['Handset Type'].fillna(df['Handset Type'].mode()[0], inplace=True)

# Aggregate information per customer
aggregated_data = df.groupby('MSISDN/Number').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode()[0]  # Get the most frequent handset type per customer
}).reset_index()

# Displaying the aggregated data
aggregated_data


,MSISDN/Number,TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP UL (kbps),Avg Bearer TP DL (kbps),Handset Type
0,3.360100e+10,2.080991e+07,759658.664811,46.000000,0.000000,39.0,37.0,Huawei P20 Lite Huawei Nova 3E
1,3.360100e+10,2.080991e+07,759658.664811,30.000000,1.000000,51.0,48.0,Apple iPhone 7 (A1778)
2,3.360100e+10,2.080991e+07,759658.664811,109.795706,17.662883,49.0,48.0,undefined
3,3.360101e+10,1.066000e+03,759658.664811,69.000000,15.000000,44.0,204.0,Apple iPhone 5S (A1457)
4,3.360101e+10,1.507977e+07,390430.332406,57.000000,2.500000,8224.5,20197.5,Apple iPhone Se (A1723)
...,...,...,...,...,...,...,...,...
106851,3.379000e+10,2.150440e+05,3001.000000,42.000000,10.000000,387.0,9978.0,Huawei Honor 9 Lite
106852,3.379000e+10,2.080991e+07,759658.664811,34.000000,6.000000,48.0,68.0,Apple iPhone 8 Plus (A1897)
106853,3.197021e+12,2.080991e+07,759658.664811,109.795706,17.662883,0.0,1.0,Quectel Wireless. Quectel Ec25-E
106854,3.370000e+14,2.080991e+07,759658.664811,109.795706,17.662883,22.0,11.0,Huawei B525S-23A
